# Elasticsearch Queries

## Setup

In [24]:
import sys
import json
sys.path.append('..')
from connectors.elk import Elk

elk = Elk()
es = elk.es

Elasticsearch: Connection sucessful


## Users not connected from more than X days

In [31]:
from datetime import datetime, timedelta

# Definición de la consulta con top_hits para obtener detalles adicionales
query = {
  "size": 0,
  "aggs": {
    "users": {
      "terms": {
        "field": "userId.keyword",
        "size": 10000  # Ajusta este número según la cantidad esperada de usuarios
      },
      "aggs": {
        "latest_login": {
          "max": {
            "field": "@timestamp"
          }
        },
        "top_user_info": {
          "top_hits": {
            "sort": [
              {
                "@timestamp": {
                  "order": "desc"
                }
              }
            ],
            "_source": {
              "includes": ["userDisplayName", "deviceDetail.displayName"]
            },
            "size": 1
          }
        }
      }
    }
  }
}

# Realiza la consulta
index_name = "logs-ms_singins"  # Asegúrate de cambiarlo por el nombre real de tu índice
result = es.search(index=index_name, body=query)

# Extrae y ordena los resultados por la fecha de última conexión de manera ascendente
users_last_login = []
for bucket in result['aggregations']['users']['buckets']:
    user_id = bucket['key']
    last_login_timestamp = bucket['latest_login']['value_as_string']
    user_info = bucket['top_user_info']['hits']['hits'][0]['_source']
    user_display_name = user_info.get('userDisplayName', 'N/A')
    device_display_name = user_info.get('deviceDetail', {}).get('displayName', 'N/A')
    users_last_login.append((user_id, user_display_name, device_display_name, last_login_timestamp))

# Ordena por la fecha de última conexión de manera ascendente
users_last_login_sorted = sorted(users_last_login, key=lambda x: x[3])

# Define el formato de fecha utilizado en tus timestamps
fecha_formato = "%Y-%m-%dT%H:%M:%S.%fZ"

# Calcula la fecha límite para los usuarios inactivos (hace más de 30 días)
limite_inactividad = datetime.now() - timedelta(days=30)

# Filtra los usuarios inactivos por más de 30 días
usuarios_inactivos = []
for user in users_last_login_sorted:
    user_id, user_display_name, device_display_name, last_login_str = user
    last_login = datetime.strptime(last_login_str, fecha_formato)

    if last_login < limite_inactividad:
        usuarios_inactivos.append(user)

# Imprime los usuarios inactivos
for user in usuarios_inactivos:
    print(user)



('8f20ce5e-0da7-432d-a91f-c04614c406c4', 'Santana, Fernando', '', '2024-01-24T20:34:50.000Z')
('77d6c11b-92a4-45c6-be2e-e6043f11ff00', 'Cabral, Ivonne', '', '2024-01-25T14:02:57.000Z')
('90042398-f05b-4405-8690-f695079ff3e4', 'tdg.galera1', '', '2024-01-25T14:22:30.000Z')
('517439f8-c726-4f42-a251-3f7de24b1015', 'Gutierres Tovar, Victor', '', '2024-01-26T09:34:00.000Z')
('78678837-7233-4487-8646-891964de3b68', 'Tabacalera Reports', '', '2024-01-29T07:56:43.000Z')
